In [ ]:
#| default_exp core

# bashnb IPython magic
> Provides `psh` persistent bash magics in Jupyter and IPython

In [ ]:
#| export
import re,os
from pathlib import Path
from IPython.core.magic import register_cell_magic
from IPython.display import display, Javascript
from IPython.paths import get_ipython_dir

In [ ]:
__file__ = './00_core.ipynb'

In [ ]:
#| export
class BashMagic:
    def __init__(self):
        self.o = ...
        self._loaded = False

    def bash(self, line, cell=None):
        if line and not cell: cell=line
        if not self._loaded:
            p = Path(__file__).resolve().parent
            self._loaded = True
        disp = True
        if cell.endswith(';'): disp,cell = False,cell[:-1]
        res = self.o.eval(cell) or None
        if disp: return res

In [ ]:
#| export
def create_magic(shell=None):
    if not shell: shell = get_ipython()
    bash_magic = BashMagic()
    shell.register_magic_function(bash_magic, 'line_cell', 'psh')

In [ ]:
# Only required if you don't load the extension
create_magic()

In [ ]:
#|export
def load_ipython_extension(ipython):
    "Required function for creating magic"
    create_magic(shell=ipython)

In [ ]:
#| export
def create_ipython_config():
    "Called by `bashnb_install` to install magic"
    ipython_dir = Path(get_ipython_dir())
    cf = ipython_dir/'profile_default'/'ipython_config.py'
    cf.parent.mkdir(parents=True, exist_ok=True)
    if cf.exists() and 'bashnb' in cf.read_text(): return print('bashnb already installed!')
    with cf.open(mode='a') as f: f.write("\nc.InteractiveShellApp.extensions.append('bashnb.core')\n\n")
    print(f"Jupyter config updated at {cf}")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()

Exception: `nbdev_export` must be called from a directory within a nbdev project.